In [3]:
"""Script to train a word tokenizer
"""

import sys, csv
from tokenizer import WordTokenizer
import pandas as pd
import keras
import os


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:

MAX_TEXTS = 1000000


In [6]:
df= pd.read_csv(r'C:\Users\Rajvi\Documents\CODING\Hackathon\Amazon ML 2021\dataset52a7b21\dataset\train_final.csv', escapechar = "\\" ,quoting = csv.QUOTE_NONE)

In [7]:
df1= df.sample(n=1000)
df1.head()

,Unnamed: 0,TITLE_clean,DESCRIPTION_clean,BULLET_POINTS_clean,MIX,BRAND,BROWSE_NODE_ID
768472,768472,avengers iron mini man figure character bundle...,marvel pack br iron man mini figure imagine in...,marvel hero series pack figure fun loot bundle...,marvel pack br iron man mini figure imagine in...,Avengers,55
620693,620693,devsang piece eyelet polyester door curtain se...,luxurious curtain made fine superior polyester...,luxurious curtain made fine superior polyester...,luxurious curtain made fine superior polyester...,Devsang,739
2432928,2432928,aldivo designer printed mobile back case cover...,aldivo designed mobile back cover implausible ...,case handmade item using state art sublimation...,aldivo designed mobile back cover implausible ...,ALDIVO,1045
909033,909033,leather retail pink colour cut sleeve girls fa...,invest pink colour jacket women leather retail...,leather retail authorized sell product seller ...,invest pink colour jacket women leather retail...,Leather Retail,13184
1286955,1286955,print station casual wear half slevees round n...,add coolness wardrobe set pace high latest tre...,fit type regular round neck stylish half sleev...,add coolness wardrobe set pace high latest tre...,PRINT STATION,5


In [8]:
df1= df1[['TITLE_clean', 'BRAND', 'MIX', 'BROWSE_NODE_ID']]


In [33]:
tags=["O"]*1000
df1['tags']=tags
df1


,title,brand,description,categories,tags
768472,avengers iron mini man figure character bundle...,avengers,marvel pack br iron man mini figure imagine in...,55,O
620693,devsang piece eyelet polyester door curtain se...,devsang,luxurious curtain made fine superior polyester...,739,O
2432928,aldivo designer printed mobile back case cover...,aldivo,aldivo designed mobile back cover implausible ...,1045,O
909033,leather retail pink colour cut sleeve girls fa...,leather retail,invest pink colour jacket women leather retail...,13184,O
1286955,print station casual wear half slevees round n...,print station,add coolness wardrobe set pace high latest tre...,5,O
...,...,...,...,...,...
2350851,micro usb cable apple android phone charger mo...,generic,material tpe abs metal function charging data ...,534,O
2660366,supremechoice car stylish drill free roof rail...,supremechoice,supremechoice car stylish drill free roof rail...,5100,O
2898418,pisara women cotton printed saree red sari,pisara,graceful gorgeous cotton saree women house pis...,1052,O
2173432,indigo womens rayon asymmetrical hemline kurta...,indigo,color name teal material rayon three quarter s...,1251,O


In [24]:
# df1.columns=["title","brand","description","categories"]
df1.head()
df1.loc[:,"brand"] = df1.brand.apply(lambda x : str.lower(str(x)))
df1.head()

,title,brand,description,categories,tags
768472,avengers iron mini man figure character bundle...,avengers,marvel pack br iron man mini figure imagine in...,55,
620693,devsang piece eyelet polyester door curtain se...,devsang,luxurious curtain made fine superior polyester...,739,
2432928,aldivo designer printed mobile back case cover...,aldivo,aldivo designed mobile back cover implausible ...,1045,
909033,leather retail pink colour cut sleeve girls fa...,leather retail,invest pink colour jacket women leather retail...,13184,
1286955,print station casual wear half slevees round n...,print station,add coolness wardrobe set pace high latest tre...,5,


In [40]:
import sys, csv

# in_file = sys.argv[1]
# out_file = '.'.join(in_file.split('.')[:-1] + ['tagged'] + ['csv'])
# with open(in_file, 'r') as f:
    # reader = csv.reader(f)
    # writer = csv.writer(open(out_file,"w"))
count = 0
for index, row in df1.iterrows():
    # print(row['tags'])
    count += 1
    if not (count % 10000):
        print (count, 'rows tagged')
    title, brand, description = str(row['title']), str(row['brand']), str(row['description'])
    tagging = ''
    brand = brand.split(' ')
    brand_started = False
    for word in title.split(' '):
        if word == brand[0]:
            tagging += 'B-B '
            brand_started = True
            # print('executing')
        elif len(brand) > 1 and brand_started:
            for b in brand[1:]:
                if word == b:
                    tagging += 'I-B '
                else:
                    brand_started = False
                    tagging += 'O '
        else:
            brand_started = False
            tagging += 'O '
    df1.at[index,'tags'] = tagging
    # writer.writerow(row)
print (count, 'rows tagged')

1000 rows tagged


In [41]:
df1['tags']

768472     B-B O O O O O O O O O O O O O O O O O O O O O 
620693                               B-B O O O O O O O O 
2432928                      B-B O O O O O O O O O O O O 
909033                             B-B I-B O O O O O O O 
1286955                  B-B I-B O O O O O O O O O O O O 
                                ...                      
2350851                              O O O O O O O O O O 
2660366                        B-B O O O O O O O O O O O 
2898418                                  B-B O O O O O O 
2173432                          B-B O O O O O O O O O O 
446726                                   B-B O O O O O O 
Name: tags, Length: 1000, dtype: object

In [42]:

# def main(df1):
    # Fetch data
texts, categories = [], []
    # with open(sys.argv[1], 'r') as f:
        # reader = csv.DictReader(f, fieldnames=["title","brand","description","categories"])
count = 0
for index, row in df1.iterrows():
    count += 1
    text, category = str(row['title'])+' '+str(row['description']), row['categories']
    texts.append(text)
    categories.append(category)
    if count >= MAX_TEXTS:
        break
print(('Processed %s texts.' % len(texts)))

# if __name__ == "__main__":
#     main(df1)


Processed 1000 texts.


In [43]:

    # Tokenize texts
    tokenizer = WordTokenizer()
    tokenizer.train(texts)


Training tokenizer...
Found 10907 unique tokens.


In [ ]:
# categories= df1.categories.unique()
# len(categories)

In [44]:
"""Script to train a product category classifier based on product titles and descriptions
"""

import sys, csv
from tokenizer import WordTokenizer
from classifier import ProductClassifier


In [ ]:

MAX_TEXTS = 1000000


In [45]:
# texts, categories = [], []
#     # with open(sys.argv[1], 'r') as f:
#         # reader = csv.DictReader(f, fieldnames=["title","brand","description","categories"])
# count = 0
# for index, row in df1.iterrows():
#     count += 1
#     text, category = str(row['title'])+' '+str(row['description']), row['categories']
#     texts.append(text)
#     categories.append(category)
#     if count >= MAX_TEXTS:
#         break
# print(('Processed %s texts.' % len(texts)))

    # Fetch data
texts, categories = [], []
# with open(sys.argv[1], 'r') as f:
        # reader = csv.DictReader(f, fieldnames=["title","brand","description","categories"])
count = 0
for index, row in df1.iterrows():
    count += 1
    text, category = str(row['description']), row['categories']
    texts.append(text)
    categories.append(category)
    if count >= MAX_TEXTS:
        break
print(('Processed %s texts.' % len(texts)))


Processed 1000 texts.


In [46]:

    # Tokenize texts
    tokenizer = WordTokenizer()
    tokenizer.load()
    data = tokenizer.tokenize(texts)


In [ ]:
# len(categories)

In [47]:

import sys, csv
from tokenizer import WordTokenizer
from classifier import ProductClassifier
from tensorflow.keras.utils import to_categorical
import numpy as np

    # Get labels from classifier
classifier = ProductClassifier()
labels = classifier.get_labels(categories)
# labels = to_categorical(np.asarray(categories))
labels.shape


Indexing categories...
Found 508 unique categories.


(1000, 508)

In [ ]:
# print(labels.shape)

In [ ]:
# print('Loading GloVe embedding...hiii')
# embeddings_index = {}
# print("hi")
# f= open("./data/glove.6B/glove.6B.100d.txt", 'r', encoding="utf-8")
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

In [48]:
classifier.compile(tokenizer)

Loading GloVe embedding...hiii
hi
Found 400000 word vectors.
Creating embedding layer...
Creating network...
Compiling network...


In [ ]:
# checkpointer = open(file=r'models/classifier.h5')
# print(checkpointer)

In [49]:
classifier.train(data, labels, epochs=2)

Training...
hi
Epoch 1/2
4/4 [==============================] - 15s 312ms/step - loss: 6.1483 - acc: 0.0637 - val_loss: 6.1350 - val_acc: 0.0950

Epoch 00001: saving model to C:\Users\Rajvi\Documents\CODING\Hackathon\Amazon ML 2021\dataset52a7b21\dataset\productner-master\productner-master\models\classifier.h5
Epoch 2/2
4/4 [==============================] - 1s 142ms/step - loss: 5.9400 - acc: 0.1013 - val_loss: 6.1064 - val_acc: 0.0900

Epoch 00002: saving model to C:\Users\Rajvi\Documents\CODING\Hackathon\Amazon ML 2021\dataset52a7b21\dataset\productner-master\productner-master\models\classifier.h5
Evaluating...
1/1 [==============================] - 0s 439ms/step
y_test= [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
predicted= [2 2 2 4 2 4 4 2 2 2 2 4 4 4 4 4 2 4 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 2 2
 2 4 2 2 2 4 2 2 2 2 2 2 2 4 2 2 2 4 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 2


C:\Users\Rajvi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Rajvi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Rajvi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

TRAIN_NER

In [51]:
"""Script to train a product category ner based on product titles and descriptions
"""
import sys, csv
from tokenizer import WordTokenizer
from ner import ProductNER

MAX_TEXTS = 1000000


    # Fetch data
texts, tags = [], []
count = 0
for index, row in df1.iterrows():
    count += 1
    text, tag_set = row['title'], row['tags'].split(' ')[:-1]
    texts.append(text)
    tags.append(tag_set)
    if count >= MAX_TEXTS:
            break
print(('Processed %s texts.' % len(texts)))


Processed 1000 texts.


In [52]:

    # Tokenize texts
    tokenizer = WordTokenizer()
    tokenizer.load()
    data = tokenizer.tokenize(texts)


In [ ]:
# from tensorflow.keras.utils import to_categorical
# from keras.preprocessing.sequence import pad_sequences
# tag_map={}
# indices = []
# for tag in tags:
#     if not (tag in tag_map):
#         tag_map[tag] = len(tag_map) + 1
#     indices.append(tag_map[tag])
# print(indices)
# labels = []
# for tag_set in tags:
#     indexed_tags = indices
#     labels.append(to_categorical(np.asarray(indexed_tags)))
# labels = pad_sequences(labels, maxlen=200)
# print(labels)     


In [53]:

    # Get labels from NER
    ner = ProductNER()
    labels = ner.get_labels(tags)


Getting labels...


In [54]:

    # Compile NER network and train
    ner.compile(tokenizer)


Loading GloVe embedding...
Found 400000 word vectors.
Creating embedding layer...
Creating network...
Compiling network...


In [55]:
ner.train(data, labels, epochs=2)



Training...
(200, 200) (800, 200) (200, 200, 4) (800, 200, 4)
(1000, 200) (1000, 200, 4)
Epoch 1/2
4/4 [==============================] - 15s 1s/step - loss: 0.0738 - acc: 0.3449 - val_loss: 0.0552 - val_acc: 0.7976

Epoch 00001: saving model to C:\Users\Rajvi\Documents\CODING\Hackathon\Amazon ML 2021\dataset52a7b21\dataset\productner-master\productner-master\models\ner.h5
Epoch 2/2
4/4 [==============================] - 2s 507ms/step - loss: 0.0537 - acc: 0.8623 - val_loss: 0.0397 - val_acc: 0.8997

Epoch 00002: saving model to C:\Users\Rajvi\Documents\CODING\Hackathon\Amazon ML 2021\dataset52a7b21\dataset\productner-master\productner-master\models\ner.h5
Evaluating...
1/1 [==============================] - 2s 2s/step
              precision    recall  f1-score   support

               0.000000  0.000000  0.000000     37854
         B-B   0.117647  0.012270  0.022222       163
           O   0.047962  0.988654  0.091486      1939
         I-B   0.000000  0.000000  0.000000        44
